## Import necessary libraries and modules

In [1]:
import pandas as pd
import numpy as np
import os 
import seaborn as sns
import statistics

In [2]:
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

I want to have interactive plots and visualizations for my data, so I'm going to use HoloViews.

In [3]:
# put all the HoloViews stuff here
# tbh might not need this? we'll see. 

### Load the cleaned dataset

In [4]:
# load in dataset
mutants = pd.read_csv('../data/interim/k8_clean_data.csv', header=None, low_memory=False)

In [5]:
mutants.head()

,0,1,2,3,4,5,6,7,8,9,...,5400,5401,5402,5403,5404,5405,5406,5407,5408,5409
0,-0.161,-0.014,0.002,-0.036,-0.033,-0.093,0.025,0.005,0.000,-0.015,...,0.013,0.021,0.020,0.016,-0.011,0.003,0.010,-0.007,a119e_l125p,inactive
1,-0.158,-0.002,-0.012,-0.025,-0.012,-0.106,0.013,0.005,0.000,-0.002,...,-0.008,0.007,0.015,-0.008,-0.011,-0.004,0.013,0.005,a119e_r283k_a353v,inactive
2,-0.169,-0.025,-0.010,-0.041,-0.045,-0.069,0.038,0.014,0.008,-0.014,...,0.010,0.025,0.025,0.021,-0.012,0.006,0.016,-0.018,c135y,inactive
3,-0.183,-0.051,-0.023,-0.077,-0.092,-0.015,0.071,0.027,0.020,-0.019,...,0.012,0.050,0.038,0.051,-0.015,0.017,0.027,-0.049,c135y_e285m,inactive
4,-0.154,0.005,-0.011,-0.013,-0.002,-0.115,0.005,0.002,-0.003,0.002,...,0.012,0.009,0.003,-0.001,0.002,-0.006,0.009,0.013,c135y_e285v,inactive


In [7]:
# pop out the column of mutants with the nametags, for safekeeping

nametags = mutants.pop(5408).astype(str)
print(nametags)

# since we'll be manipulating the columns of the mutants dataframe, we don't need to keep track of the labels associated with each row

0              a119e_l125p
1        a119e_r283k_a353v
2                    c135y
3              c135y_e285m
4              c135y_e285v
               ...        
16586    y220c_t230c_n239y
16587    y220c_y234f_n239l
16588                y234c
16589          y234c_a119e
16590          y234f_n239l
Name: 5408, Length: 16591, dtype: object


In [8]:
# new mutants dataframe
mutants.head()

,0,1,2,3,4,5,6,7,8,9,...,5399,5400,5401,5402,5403,5404,5405,5406,5407,5409
0,-0.161,-0.014,0.002,-0.036,-0.033,-0.093,0.025,0.005,0.000,-0.015,...,0.006,0.013,0.021,0.020,0.016,-0.011,0.003,0.010,-0.007,inactive
1,-0.158,-0.002,-0.012,-0.025,-0.012,-0.106,0.013,0.005,0.000,-0.002,...,0.002,-0.008,0.007,0.015,-0.008,-0.011,-0.004,0.013,0.005,inactive
2,-0.169,-0.025,-0.010,-0.041,-0.045,-0.069,0.038,0.014,0.008,-0.014,...,0.019,0.010,0.025,0.025,0.021,-0.012,0.006,0.016,-0.018,inactive
3,-0.183,-0.051,-0.023,-0.077,-0.092,-0.015,0.071,0.027,0.020,-0.019,...,0.051,0.012,0.050,0.038,0.051,-0.015,0.017,0.027,-0.049,inactive
4,-0.154,0.005,-0.011,-0.013,-0.002,-0.115,0.005,0.002,-0.003,0.002,...,-0.011,0.012,0.009,0.003,-0.001,0.002,-0.006,0.009,0.013,inactive


## Dimensionality Reduction

Due to the highly dimensional nature of this dataset (5408 features!!) and the resulting complexity, knowing where to start is difficult. More common visualization techniques for lower dimensional data are too computationally expensive and time-consuming to be useful at this point. 

Some first steps to reduce dimensionality would be:

1) Search for features with no variance and drop these features, since they do not provide any information for our classification problem.

2) Look for multicollinearity in the features! Identify features that are highly linearly correlated with each other or features that have a high PMI (point-wise mutual information) score. These features are redundant, as having many features with a high PMI score or high linear correlation do not provide additional information. 

Since I am not sure yet whether the features have linear or nonlinear (geometric?) relationships, I will be using both the linear correlation and the PMI scores (mutual information captures non-linear relationships) for the dimensionality reduction and comparing the results of both methods.

### Find Features with No Variance or Low Variance

Using the pandas describe() method, I will be looking for numerical features that have a standard deviation = 0 and min and max values that are equal to each other. These features will be dropped. 

I will also make a note of features that have very low variance, especially compared to the rest of the features, and investigate the relevance of those features further.

In [11]:
desc = mutants.describe()
desc.head()

,0,1,2,3,4,5,6,7,8,9,...,5398,5399,5400,5401,5402,5403,5404,5405,5406,5407
count,16591.000000,16591.000000,16591.000000,16591.000000,16591.000000,16591.000000,16591.000000,16591.000000,16591.000000,16591.000000,...,16591.000000,16591.000000,16591.000000,16591.000000,16591.000000,16591.000000,16591.000000,16591.000000,16591.000000,16591.000000
mean,-0.201763,-0.004898,-0.011593,-0.024726,-0.019615,-0.077004,0.222236,0.011294,0.009716,0.009895,...,-0.009736,0.002827,0.004689,0.011400,0.018242,0.001767,-0.011238,0.001415,0.011424,-0.000615
std,0.415089,0.362149,0.244942,0.256497,0.195207,0.539291,1.991421,0.127339,0.210814,0.339393,...,0.011281,0.020840,0.018827,0.024245,0.013061,0.033117,0.009405,0.015348,0.009534,0.022943
min,-6.085000,-7.409000,-4.410000,-3.419000,-3.270000,-2.241000,-0.512000,-0.685000,-0.684000,-0.534000,...,-0.392000,-0.167000,-0.233000,-0.531000,-0.075000,-0.204000,-0.522000,-0.042000,-0.112000,-0.194000
25%,-0.169000,-0.024000,-0.014000,-0.040000,-0.043000,-0.113000,0.006000,0.002000,-0.003000,-0.017000,...,-0.014000,-0.008000,-0.007000,-0.002000,0.011000,-0.019000,-0.012000,-0.006000,0.009000,-0.014000


Before I start removing features, I want to explore the standard deviations of these features. What are the ranges of the standard deviations? What is the smallest standard deviation? What is the largest standard deviation? What is the mean and median standard deviation? Are any of the standard deviations equal to 0? 

Doing this will give me a better idea of which features have relatively low variance and won't be useful.

In [14]:
desc.index

Index(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'], dtype='object')

In [17]:
descT = desc.T
descT.head()

,count,mean,std,min,25%,50%,75%,max
0,16591.0,-0.201763,0.415089,-6.085,-0.169,-0.162,-0.154,2.323
1,16591.0,-0.004898,0.362149,-7.409,-0.024,-0.010,0.005,4.552
2,16591.0,-0.011593,0.244942,-4.410,-0.014,-0.010,-0.003,2.872
3,16591.0,-0.024726,0.256497,-3.419,-0.040,-0.034,-0.012,17.831
4,16591.0,-0.019615,0.195207,-3.270,-0.043,-0.023,-0.001,4.571


In [33]:
std_min = descT['std'].min()
print("The minimum standard deviation for the original features is {}".format(std_min))

std_max = descT['std'].max()
print("The maximum standard deviation for the original features is {}".format(std_max))

# plot a histogram of the descT['std'] values so that we can see the distribution
sns.histplot(data=descT, x="std")

The minimum standard deviation for the original features is 0.006618814893812356
The maximum standard deviation for the original features is 77.3480716547277


AttributeError: module 'seaborn' has no attribute 'histplot'

In [ ]:
# write a loop to remove features with no variance first

for col in desc:
    feature = desc[col] # column contents as a Series
    
    if feature[]

### Using Linear Correlation to Reduce Dimensionality

My strategy for dimensionality reduction using linear correlation, excluding non-numerical features, is as follows:

1) Start by looking at only the 2D, or electrostatic and surface-based features. This will be the first 4827 features, or columns, in the dataset.

2) Calculate the linear correlation of the first two features with each other. This is a measure of the **redundancy** of the two features.
    
    (a) If this value is higher than the chosen threshold value, then that tells me that the two features are collinear and are redundant. Now, decide which of the two features to drop by proceeding to step #3.
    
    (b) If this value is lower than the chosen threshold value, then that tells me that the two features are not collinear and both features will be retained in the dataset. Proceed to step #4.

3) Take the two features from step #2 and calculate their correlation with the rest of the features. This will give a [2 x 4827] correlation matrix. Calculate the median of the values of each row, giving two numbers which summarize how highly correlated the two features are with the rest of the 2D features, aka **the relevancy of that feature to the rest of the dataset**. *The average is susceptible to bias from outliers, so the median is a better summary statistic* 
    
    (a) If one of the two values is much larger than the other, then drop the feature with the larger value since that feature has more repeated information over the whole dataset than the other feature. If the two values are the same, then retain both features in the dataset.

4) Move on to the next two features and repeat the process for all of the 2D features. 

5) After subsetting the 2D features in this way, repeat the process with the 3D (distance-based) features and combine the data. 

6) Investigate the relationships of the selected features with the target variable (inactive or active). 

### Using Mutual Information to Reduce Dimensionality

My strategy for dimensionality reduction using mutual information is as follows:

1) Determine a threshold value for the PMI score. 

2) Take the first two features (columns) and calculate the PMI score between them. 

    (a) If the PMI score is large (above threshold value), then that means that those two features are highly dependent. In this case, keep the feature with the largest information gain and store in a dataframe. The other feature will be removed and stored in a separate dataframe. 
    
    (b) If the PMI score is low (below threshold value), then those two features are weakly correlated and both features may be retained. 
    
    (c) If the PMI score is 0, then the two features are statistically independent and both features may be retained.


3) Repeat the process with the next set of two features until all the 2D features have been tested. 

4) Continue the process with the 3D features.

Afterwards, I will investigate the relationships of the remaining features with the target variable.